In [1]:
from general_trader import GeneralTrader
import polars as pl
import polars_talib as pltalib
from trading_helpers import TradingData

In [3]:
traderData = TradingData(
    "/mnt/jonarne/dev/trading_data/",
    "/SP500/",
    "SP500_list.parquet",
)
# download_all_tickers. This function is only needed to call once,
# or when you decide to download new fresh ticker and sector data
#traderData.download_all_tickers()


# populate polars dataframe with open, close, high, low and volume data
ochl_data = traderData.load_ticker_parquet_files()
print(ochl_data.collect())


TradingData Initialized
shape: (4_300_704, 7)
┌─────────────────────┬───────────┬────────────┬───────────┬───────────┬─────────┬────────┐
│ date                ┆ close     ┆ high       ┆ low       ┆ open      ┆ volume  ┆ ticker │
│ ---                 ┆ ---       ┆ ---        ┆ ---       ┆ ---       ┆ ---     ┆ ---    │
│ datetime[ns]        ┆ f64       ┆ f64        ┆ f64       ┆ f64       ┆ i64     ┆ str    │
╞═════════════════════╪═══════════╪════════════╪═══════════╪═══════════╪═════════╪════════╡
│ 1994-10-18 00:00:00 ┆ 1.205047  ┆ 1.214462   ┆ 1.186218  ┆ 1.205047  ┆ 418166  ┆ O      │
│ 1994-10-19 00:00:00 ┆ 1.308607  ┆ 1.318021   ┆ 1.214463  ┆ 1.214463  ┆ 249950  ┆ O      │
│ 1994-10-20 00:00:00 ┆ 1.318019  ┆ 1.365091   ┆ 1.318019  ┆ 1.327434  ┆ 141590  ┆ O      │
│ 1994-10-21 00:00:00 ┆ 1.299191  ┆ 1.327434   ┆ 1.289776  ┆ 1.318019  ┆ 68731   ┆ O      │
│ 1994-10-24 00:00:00 ┆ 1.270948  ┆ 1.327435   ┆ 1.270948  ┆ 1.299192  ┆ 75749   ┆ O      │
│ …                   ┆ …         

In [5]:


    # Define a sample Polars expression
    #my_filter = (pl.col("close") > 100) & (pl.col("volume") > 50000)
my_filter = None
    # --- Test Cases ---

print("--- Testing Valid Instantiation ---")
try:
    trader = GeneralTrader(ohlcColumnsToUse=["close"], df_ohlc=ochl_data.collect(), initialFilter=my_filter)
    print("Successfully created GeneralTrader instance.")
    print("DataFrame Schema:", trader.df_ohlc.schema)
    print("Initial Filter:", trader.initialFilter)
except ValidationError as e:
    print("Validation Error (unexpected):", e)

--- Testing Valid Instantiation ---
Successfully created GeneralTrader instance.
DataFrame Schema: Schema([('ticker', String), ('date', Datetime(time_unit='ns', time_zone=None)), ('close', Float64)])
Initial Filter: None


In [6]:
trader.df_ohlc

ticker,date,close
str,datetime[ns],f64
"""O""",1994-10-18 00:00:00,1.205047
"""O""",1994-10-19 00:00:00,1.308607
"""O""",1994-10-20 00:00:00,1.318019
"""O""",1994-10-21 00:00:00,1.299191
"""O""",1994-10-24 00:00:00,1.270948
…,…,…
"""NRG""",2025-04-11 00:00:00,94.470001
"""NRG""",2025-04-14 00:00:00,95.919998
"""NRG""",2025-04-15 00:00:00,97.110001


In [8]:
indicators = {
    "sma": pltalib.sma(timeperiod=200).over("ticker"),
    "mean_B": pl.col("close").mean().over("ticker"),
    #"bad_stuff": 5,
    "A_plus_B": pl.col("close") + pl.col("close"),
}
trader.add_columns(indicators)

GeneralTrader(df_ohlc=shape: (4_300_704, 6)
┌────────┬─────────────────────┬───────────┬───────────┬───────────┬────────────┐
│ ticker ┆ date                ┆ close     ┆ sma       ┆ mean_B    ┆ A_plus_B   │
│ ---    ┆ ---                 ┆ ---       ┆ ---       ┆ ---       ┆ ---        │
│ str    ┆ datetime[ns]        ┆ f64       ┆ f64       ┆ f64       ┆ f64        │
╞════════╪═════════════════════╪═══════════╪═══════════╪═══════════╪════════════╡
│ O      ┆ 1994-10-18 00:00:00 ┆ 1.205047  ┆ NaN       ┆ 22.008229 ┆ 2.410094   │
│ O      ┆ 1994-10-19 00:00:00 ┆ 1.308607  ┆ NaN       ┆ 22.008229 ┆ 2.617214   │
│ O      ┆ 1994-10-20 00:00:00 ┆ 1.318019  ┆ NaN       ┆ 22.008229 ┆ 2.636039   │
│ O      ┆ 1994-10-21 00:00:00 ┆ 1.299191  ┆ NaN       ┆ 22.008229 ┆ 2.598381   │
│ O      ┆ 1994-10-24 00:00:00 ┆ 1.270948  ┆ NaN       ┆ 22.008229 ┆ 2.541897   │
│ …      ┆ …                   ┆ …         ┆ …         ┆ …         ┆ …          │
│ NRG    ┆ 2025-04-11 00:00:00 ┆ 94.470001 ┆ 90.291616